In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
    database = 'postgres_db',
    user = 'postgres_user',
    password = 'admin',
    host = 'client.tuple-mia.com',
    port = '5432',
    sslmode = 'require')

In [3]:
temp = conn.cursor()

In [4]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml','r') as f:
    doc = yaml.load(f)

In [5]:
print doc['table_map']
print doc['column_map']

{'CUSTOMER_CONTACT': 'contact', 'EVENT_LOG': 'event', 'TRANSACTION_MASTER': 'trans', 'CUSTOMER_SECONDARY': 'temp_customer_secondary', 'EVENT_MASTER': 'event_master', 'CUSTOMER_MASTER': 'users'}
{'CUSTOMER_CONTACT': {'phone_number': 'phone', 'cust_id': 'id', 'firstname': 'first_name', 'facebook_id': 'facebook_id', 'email_id': 'email', 'lastname': 'last_name'}, 'EVENT_LOG': {'timestamp': 'timestamp', 'cust_id': 'from_id', 'product_id': 'to_id', 'action_type': 'actiontype'}, 'TRANSACTION_MASTER': {'cust_id': 'userid', 'product_id': 'product_id', 'revenue': 'revenue', 'timestamp': 'purchase_date', 'renewal': 'renewal', 'quantity': 'quantity'}, 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'}, 'EVENT_MASTER': {'action_category_id': None, 'action_category_name': None, 'action_type': 'actiontype', 'action_name': 'action_desc'}, 'CUSTOMER_MASTER': {'create_date': 'create_date', 'birthday': 'birthday', 'cust_id': 'id'}}


In [6]:
query = 'SELECT * FROM %s'%(doc['table_map']['EVENT_LOG'])

In [7]:
print query

SELECT * FROM event


In [8]:
temp.execute(query)

In [9]:
event = temp.fetchall()

In [10]:
len(event)

7163170

In [11]:
col_names = [desc[0] for desc in temp.description]

In [12]:
col_names

['to_id', 'timestamp', 'actiontype', 'from_id']

In [13]:
import pandas as pd

In [14]:
event_table = pd.DataFrame(event, columns=col_names)

In [15]:
event_table.head()

,to_id,timestamp,actiontype,from_id
0,255074,2016-11-19 00:52:13+00:00,1,6148771
1,304603,2016-11-20 08:03:32+00:00,1,6148771
2,368895,2016-11-15 09:28:35+00:00,1,6148771
3,376784,2016-11-15 09:43:29+00:00,1,6148771
4,462976,2016-11-15 09:44:27+00:00,1,6148771


In [16]:
event_table.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace=True)
event_table.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace=True)
event_table.rename(columns={doc['column_map']['EVENT_LOG']['action_type']:'action_type'}, inplace=True)

In [17]:
event_table.head()

,product_id,timestamp,action_type,cust_id
0,255074,2016-11-19 00:52:13+00:00,1,6148771
1,304603,2016-11-20 08:03:32+00:00,1,6148771
2,368895,2016-11-15 09:28:35+00:00,1,6148771
3,376784,2016-11-15 09:43:29+00:00,1,6148771
4,462976,2016-11-15 09:44:27+00:00,1,6148771


In [18]:
event_df = event_table[['cust_id', 'product_id', 'action_type']]

In [19]:
event_df.head()

,cust_id,product_id,action_type
0,6148771,255074,1
1,6148771,304603,1
2,6148771,368895,1
3,6148771,376784,1
4,6148771,462976,1


In [20]:
len(event_df.cust_id.unique())

6256

In [21]:
len(event_df.product_id.unique())

386251

In [22]:
grp_cust = event_df.groupby(['cust_id']).count()

In [23]:
grp_cust.head()

,product_id,action_type
cust_id,,
3289765,683,683
6148771,488,488
6148821,7077,7077
6148833,1,1
6149073,287,287


In [ ]:
grp_cust= grp_cust.reset_index()

In [ ]:
grp_cust.rename(columns = {'product_id':'value'}, inplace=True)

In [ ]:
grp_cust.head()

In [ ]:
grp_cust['value'].max()

In [ ]:
grp_cust_cou = grp_cust.groupby(['value']).count()

In [ ]:
grp_cust_cou.head()

In [ ]:
grp_cust_cou = grp_cust_cou.reset_index()

In [ ]:
grp_cust_cou.rename(columns = {'value':'no_of_products'}, inplace=True)
grp_cust_cou.rename(columns = {'cust_id':'users_count'}, inplace=True)

In [ ]:
grp_cust_cou.head()

In [ ]:
grp_cust_cou.tail()

In [ ]:
grp_cust_cou['no_of_products'].max()

In [ ]:
reversed_df = grp_cust_cou.sort_index(ascending = False)

In [ ]:
reversed_df.head()

In [ ]:
reversed_df['users_per'] = ""

In [ ]:
reversed_df['aggregated_per'] = ""

In [ ]:
reversed_df.head()

In [ ]:
reversed_df['users_count'].sum()

In [ ]:
reversed_df['users_per'] = reversed_df['users_count']/reversed_df['users_count'].sum() * 100

In [ ]:
reversed_df.head()

In [ ]:
reversed_df['aggregated_per'] = reversed_df.users_per.cumsum()

In [ ]:
reversed_df.head()

In [ ]:
cust_df = reversed_df

In [ ]:
cust_df.head()

In [ ]:
cust_df.tail(30)

In [ ]:
grp_cust.head()

In [ ]:
grp_cust.ix[0,0]

In [ ]:
cust = grp_cust.loc[grp_cust['value'] > 28]

In [ ]:
del cust['value']

In [ ]:
cust.head()

In [ ]:
cust.shape

In [ ]:
grp_cust.head()